# Shared data directory (Experimental)

In this tutorial we will describe a shared directory `data`. Qiskit patterns can produce and consume files in the `data` directory.  The files in the `data` directory are shared among Qiskit patterns.

`QiskitServerless` has `file_download`, `file_upload`, `file_delete` and `files` functions that provide file upload, file download, file delete and list files function.

Limitations:

- only `tar` and `h5` files are supported
- `tar` or `h5`file should be saved in `/data` director during your pattern execution to be visible by `.files()` method call
- only `/data` directory is supported, `/data/other_folder` will not be visible
- The working directory of these functions are `/data`

> &#x26A0; This interface is experimental, therefore it is subjected to breaking changes.

> &#x26A0; This provider is set up with default credentials to a test cluster intended to run on your machine. For information on setting up infrastructure on your local machine, check out the guide on [local infrastructure setup](https://qiskit.github.io/qiskit-serverless/deployment/local.html).

In [1]:
import os
from qiskit_serverless import ServerlessClient, QiskitFunction

serverless = ServerlessClient(
    token=os.environ.get("GATEWAY_TOKEN", "awesome_token"),
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)
serverless

<ServerlessProvider: gateway-provider>

Upload a file in the `data` directory

In [2]:
import tarfile

filename= "uploaded_file.tar"
file= tarfile.open(filename,"w")
file.add("manage_data_directory.ipynb")
file.close() 

In [3]:
serverless.file_upload(filename)

'{"message":"/usr/src/app/media/mockuser/uploaded_file.tar"}'

Produce a file in the `data`directory

In [4]:
function = QiskitFunction(
    title="file-producer", entrypoint="produce_files.py", working_dir="./source_files/"
)

serverless.upload(function)
job = serverless.run("file-producer")
job.result()

{'Message': 'my_file.txt archived into my_file.tar'}

List files in the `data` directory

In [5]:
serverless.files()

['uploaded_file.tar', 'my_file.tar']

Consume the file procuded by `file-producer` Qiskit Pattern

In [6]:
function = QiskitFunction(
    title="file-consumer", entrypoint="consume_files.py", working_dir="./source_files/"
)

serverless.upload(function)
job = serverless.run("file-consumer")
job.result()

{'Message': "b'Hello!'"}

List files

In [8]:
serverless.files()

['uploaded_file.tar', 'my_file.tar']

Delete files

In [9]:
serverless.file_delete("uploaded_file.tar")

'Requested file was deleted.'